# Build a copilot using Azure AI Studio

In this tutorial, you will learn how to build, run, and evaluate, a copilot application using your own custom data.


**1 | Pre-Work: Setup AI Project**

Before opening this Notebook, we assume you already completed the following steps:

1. [Step 1](./../docs/step-01.md) - setup your development environment.
1. [Step 2](./../docs/step-02.md) - run `ai init` to create AI project
1. [Step 3a](./../docs/step-03.md) - run `ai search` to create Search index
1. [Step 3b](./../docs/step-03.md) - run `ai dev new .env` to save env vars.

🎉 _Congratulations!_ - You are now ready to start working with the Azure AI Studio SDK to interact with your Azure AI resources!

**2 | Setup: Import Required Libraries**

In [ ]:
# Load environment variables (from .env)
from dotenv import load_dotenv
load_dotenv()

**3 | Define: Azure AI Search Helper Function**

In [ ]:
## Helper Function 1 => GET INDEXED DOCUMENTS FOR QUERY
## Given a query (question)and a desired number of documents (default=5)
## Retrieve the relevant set of matching results for the query from Cognitive Search

import os
import openai
from azure.search.documents.aio import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import RawVectorQuery

async def get_documents(query, num_docs=5):

    #  retrieve documents relevant to the user's question from Cognitive Search
    search_client = SearchClient(
        endpoint=os.environ["AZURE_AI_SEARCH_ENDPOINT"],
        credential=AzureKeyCredential(os.environ["AZURE_AI_SEARCH_KEY"]),
        index_name=os.environ["AZURE_AI_SEARCH_INDEX_NAME"])

    # generate a vector embedding of the user's question
    embedding = await openai.Embedding.acreate(input=query,
                                               model=os.environ["AZURE_OPENAI_EMBEDDING_MODEL"],
                                               deployment_id=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"])
    embedding_to_query = embedding["data"][0]["embedding"]

    # initialize context, then update it with results from query
    context = ""
    async with search_client:

        # use the vector embedding to do a vector search on the index
        vector_query = RawVectorQuery(vector=embedding_to_query, k=num_docs, fields="contentVector")
        results = await search_client.search(
            search_text="",
            vector_queries=[vector_query],
            select=["id", "content"])

        async for result in results:
            context += f"\n>>> From: {result['id']}\n{result['content']}"
    return context

**4 | Define Azure OpenAI ChatCompletion Helper Function**

In [ ]:
## Helper Function 2 => GET CHAT COMPLETION MESSAGES
## Given a list of user messages, a streaming flag, session state and context
##    Call get_documents to get 5 relevant results matching the user question
##    Make copy of context and modify to add retrieved documents
##    Add retrieved documents to the system prompt section
##    Call Azure OpenAI with the system prompt and user question

import jinja2
import pathlib

from streaming_utils import add_context_to_streamed_response

templateLoader = jinja2.FileSystemLoader(searchpath="./")
templateEnv = jinja2.Environment(loader=templateLoader)
system_message_template = templateEnv.get_template("system-message.jinja2")

async def chat_completion(messages: list[dict], stream: bool = False,
                          session_state: any = None, context: dict[str, any] = {}):
    # get search documents for the last user message in the conversation
    user_message = messages[-1]["content"]
    documents = await get_documents(user_message, context.get("num_retrieved_docs", 5))

    # make a copy of the context and modify it with the retrieved documents
    context = dict(context)
    context['documents'] = documents

    # add retrieved documents as context to the system prompt
    system_message = system_message_template.render(context=context)
    messages.insert(0, {"role": "system", "content": system_message})

    # call Azure OpenAI with the system prompt and user's question
    response = openai.ChatCompletion.create(
        engine=os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT"),
        messages=messages, temperature=context.get("temperature", 0.7),
        stream=stream,
        max_tokens=800)

    # add context in the returned response
    if not stream:
        response.choices[0]['context'] = context
    else:
        response = add_context_to_streamed_response(response, context)
    return response

**5 | Invoke the Chat Completion Endpoint with the User Question**

In [ ]:
# Implementation = "aisdk"  (default only)
# For now just invoke the method in the default python script.
from chat import chat_completion

# Simple question test
question = "Which tent is the most waterproof?"

# Define an async function to call the chat function
result = await chat_completion([{"role": "user", "content": question}], stream=False)
print(result)